# 实验 3：不同观测噪声水平下的 POMDP 求解效果分析

---

## 🧪 实验目标

在本实验中，我们研究 POMCP 算法在 **不同观测噪声水平** 下的表现。

我们关注的核心问题：

- 随着观测噪声变大（信念不准确），POMCP 的决策效果是否下降？
- 信念不准时是否会导致 agent 迷失方向或停滞？

---

## 📊 指标衡量：

- 成功率 `Success Rate`
- 平均奖励 `Avg Reward`
- 平均步数 `Avg Steps`
- 平均规划时间 `Avg Time per Episode`

---


In [ ]:
from core import PartialObservableGrid, POMCPAgent
from utils.base_experiment import BaseExperiment
from utils.metrics import compute_metrics, plot_metric_across_configs
from utils.utils import set_seed
import os


In [ ]:
set_seed(42)
os.makedirs("results", exist_ok=True)

# 测试不同的观测噪声水平
obs_noise_levels = [0.1, 0.3, 0.7, 1.0, 1.5]

N_EPISODES = 50

config_list = []

for noise_std in obs_noise_levels:
    config = {
        "seed": 42,
        "num_episodes": N_EPISODES,
        "max_steps": 60,
        "output_file": f"results/exp3_pomcp_noise_{noise_std}.json",
        "env_args": {
            "grid_size": (5, 5),
            "start_pos": (0, 0),
            "goal_pos": (4, 4),
            "obstacles": {(2, 2)},
            "obs_noise_std": noise_std
        },
        "agent_args": {
            "belief_type": "gridworld",
            "n_simulations": 300,
            "max_depth": 25,
            "num_particles": 200
        }
    }
    config_list.append(config)


In [ ]:
results = []

for i, config in enumerate(config_list):
    print(f"▶️ Running config {i} with obs_noise_std = {config['env_args']['obs_noise_std']}")
    exp = BaseExperiment(
        env_class=PartialObservableGrid,
        agent_class=POMCPAgent,
        config=config
    )
    metrics = exp.run()
    results.append(metrics)
    print(f"✅ Result {i}: {metrics}\n")


In [ ]:
labels = [f"σ={cfg['env_args']['obs_noise_std']}" for cfg in config_list]
file_paths = [cfg["output_file"] for cfg in config_list]

# 成功率
plot_metric_across_configs(
    file_paths=file_paths,
    labels=labels,
    metric_name="success_rate",
    ylabel="Success Rate",
    title="POMCP Success Rate vs Observation Noise"
)

# 平均奖励
plot_metric_across_configs(
    file_paths=file_paths,
    labels=labels,
    metric_name="avg_reward",
    ylabel="Average Reward",
    title="POMCP Reward vs Observation Noise"
)

# 平均步数
plot_metric_across_configs(
    file_paths=file_paths,
    labels=labels,
    metric_name="avg_steps",
    ylabel="Average Steps",
    title="POMCP Steps vs Observation Noise"
)

# 平均规划时间
plot_metric_across_configs(
    file_paths=file_paths,
    labels=labels,
    metric_name="avg_time",
    ylabel="Average Planning Time (s)",
    title="POMCP Planning Time vs Observation Noise"
)


## 📈 实验结果分析

- 当观测噪声 `σ` 很小时，智能体能准确判断自身位置，信念更新效果良好。
- 噪声上升后，POMCP 性能显著下降，成功率与平均奖励明显降低。
- 步数与规划时间受限于最大步骤和模拟次数控制，变化不大。
- 在高噪声下，agent 更可能偏离路径、陷入无效区域。

---

## 🔎 思考与改进

- 是否可以使用更强的 rollout 策略，缓解信念不准的影响？
- 粒子数是否足够？是否可以自适应增减？
- 高噪声场景下是否应该使用近似方法（如 Q-MDP）？

---
